In [1]:
# Imports

from modules import adquire, cleaning, create_df_checking, outputs


def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,_= create_df_checking(df_students, df_katas)    
    outputs(df,False)


if __name__ == '__main__':
    main()

juliopdata presentará la kata give-me-a-diamond


slug,valid-phone-number,give-me-a-diamond
username,,
Maika7,True,True
Ovión,True,True
mariadrados,True,True
jlmingo,True,False
seorakwon,True,False
ladyCelia,True,False
castares,True,False
jgph91,True,False
Clapiniella,True,False
